3.)	What are your recommendations for total number of scooters for the city overall and density of scooters by zip code?

Tasked with creating and joining geospatial data for group project to be used with Tableau for analysis.

In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [2]:
zipcodes = gpd.read_file('../data/zipcodes.geojson') ### Reading in geojson for multipolygon options.
print(zipcodes.crs)
zipcodes.head( )

epsg:4326


,zip,objectid,po_name,shape_stlength,shape_starea,geometry
0,37115,1,MADISON,178783.02488886821,596553400.57885742,"MULTIPOLYGON (((-86.68725 36.31821, -86.68722 ..."
1,37216,3,NASHVILLE,75820.997821400058,188884682.28344727,"MULTIPOLYGON (((-86.73451 36.23774, -86.73425 ..."
2,37204,9,NASHVILLE,93180.292250425613,200664795.51708984,"MULTIPOLYGON (((-86.77914 36.13424, -86.77923 ..."
3,37027,11,BRENTWOOD,159760.6942933173,174978422.04101562,"MULTIPOLYGON (((-86.81258 36.06319, -86.81263 ..."
4,37064,18,FRANKLIN,28995.828320601937,46969608.005737305,"MULTIPOLYGON (((-87.02197 36.01200, -87.02140 ..."


In [3]:
distinct_scooters_geo = pd.read_csv('../data/distinct_scooters_geo.csv') ### bringing in project data to merge.
print(distinct_scooters_geo.shape)
distinct_scooters_geo.head(3)

(10018, 4)


,sumdid,company,lat,lon
0,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,Bolt,36.121115,-86.814587
1,Powered-01a24436-0315-e1bb-7ce0-d081d05dff7d,Bolt,36.140538,-86.814606
2,Powered-03be23ca-d43b-222f-be54-e44b5b4690df,Bolt,36.136750,-86.813831


In [5]:
distinct_scooters_geo['geometry'] = distinct_scooters_geo.apply(lambda x: Point((float(x.lon),
                                                         float(x.lat))), ### adding geometry column to convert to geodf.
                                        axis=1)
distinct_scooters_geo.head(3)

C:\Users\patri\Documents\AnacondaNav\envs\geospatial_ds6\lib\site-packages\pandas\core\dtypes\cast.py:130: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,sumdid,company,lat,lon,geometry
0,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,Bolt,36.121115,-86.814587,POINT (-86.814587 36.121115)
1,Powered-01a24436-0315-e1bb-7ce0-d081d05dff7d,Bolt,36.140538,-86.814606,POINT (-86.814606 36.140538)
2,Powered-03be23ca-d43b-222f-be54-e44b5b4690df,Bolt,36.136750,-86.813831,POINT (-86.813831 36.13675)


In [6]:
distinct_scooters_geo = gpd.GeoDataFrame(distinct_scooters_geo,  ### converting to geodataframe for geospatial join.
                           crs = zipcodes.crs, 
                           geometry = distinct_scooters_geo['geometry'])

In [7]:
scooters_by_zip = gpd.sjoin(distinct_scooters_geo, zipcodes, op = 'within') ### geospatial join is ready

C:\Users\patri\Documents\AnacondaNav\envs\geospatial_ds6\lib\site-packages\IPython\core\interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [10]:
scooters_by_zip.head() ###testing to make sure things are how I want them.

,sumdid,company,lat,lon,geometry,index_right,zip,objectid,po_name,shape_stlength,shape_starea
0,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,Bolt,36.121115,-86.814587,POINT (-86.81459 36.12112),44,37215,38,NASHVILLE,158698.30754566603,424774478.15600586
4,Powered-0479bb84-afbd-0426-f1c4-df628542a88c,Bolt,36.122782,-86.815677,POINT (-86.81568 36.12278),44,37215,38,NASHVILLE,158698.30754566603,424774478.15600586
8,Powered-05fedb39-a00f-a5e0-2219-8c3c7dd1b80e,Bolt,36.120917,-86.814540,POINT (-86.81454 36.12092),44,37215,38,NASHVILLE,158698.30754566603,424774478.15600586
11,Powered-09c75f98-6492-9d35-12e4-c5649cebf403,Bolt,36.125102,-86.815059,POINT (-86.81506 36.12510),44,37215,38,NASHVILLE,158698.30754566603,424774478.15600586
13,Powered-0ac70fa5-9f26-e69b-ca7a-9803d2aa1535,Bolt,36.123545,-86.814578,POINT (-86.81458 36.12355),44,37215,38,NASHVILLE,158698.30754566603,424774478.15600586


In [8]:
scooters_full_table = scooters_by_zip ### renaming to export to csv by requested file name.

In [9]:
scooters_full_table.to_csv('scooters_full_table.csv')

In [10]:
scooters_valuecount = scooters_full_table['zip'].value_counts()

In [11]:
test_joining = scooters_full_table.groupby('zip')['sumdid'].sum()

In [12]:
test_joining.info

<bound method Series.info of zip
37013             Powered7OHBXNSZHZNNAPoweredDJX737TJBYCVY
37027    Powered218536Powered2QTOFIQINRR6MPowered342HMP...
37064                                         PoweredPR9SN
37115                             Powered28NX9PoweredI946K
37143             Powered2ZA4GQSKRFQEOPoweredAAFJRQCM2BIJR
37201    Powered09437444-bee2-57f0-81d8-a14d49b2da8fPow...
37203    Powered-0540c92c-0907-dafa-b19f-011495068ec5Po...
37204    Powered-1e5cf2e7-11c5-06d3-9bb4-3497eeaab2bcPo...
37205    Powered-046201fb-6532-1f37-6334-3612fb1e61f7Po...
37206    Powered009677Powered055493Powered171208Powered...
37207                         Powered4969809Powered6499832
37208    Powered0b2668e4-8555-5fbe-b2af-c4ddca392628Pow...
37209    Powered-0a8e2ca5-5eae-511e-ef0d-74d582fa8ceePo...
37210    Powered-ff728c28-5239-68af-ddf3-6d1207b01196Po...
37211    Powered00746183-96c8-58d7-97b5-04a091c91b34Pow...
37212    Powered-01a24436-0315-e1bb-7ce0-d081d05dff7dPo...
37213    Powered367a8b7

In [13]:

zip_company_counts = scooters_by_zip.groupby('zip')['company'].value_counts().reset_index(name='count')

# Printing the resulting table of scooters and their respective companies sorted by zipcode.
print(zip_company_counts)


       zip company  count
0    37013    Lime      2
1    37027    Lime     31
2    37027    Jump      7
3    37027    Bird      6
4    37027  Gotcha      2
..     ...     ...    ...
105  37232    Spin      1
106  37240    Bird     13
107  37240    Spin      5
108  37240    Lyft      4
109  37240    Lime      2

[110 rows x 3 columns]


In [14]:
zip_company_counts.to_csv('zip_company_counts.csv')